<a href="https://colab.research.google.com/github/RamsesMDLC/LLM-Fine-Tuning/blob/main/LLM_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LLM Fine-Tuning (Chat Template)

In [4]:
#API key
  #Provides a secure way to access stored secrets (like API tokens) within Google Colab.
from google.colab import userdata
  #Allows programmatic login to Hugging Face Hub.
from huggingface_hub import login

In [1]:
#Pipeline:
  #Function or abstraction from the Hugging Face Transformers library.
  #It is a high-level API for easy access to various NLP tasks (in this case, manage "chat templates automatically")
from transformers import pipeline

In [5]:
pipe1 = pipeline ("text-generation","HuggingFaceTB/SmolLM3-3B",device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [3]:
2+2

4